# Парсер биржи фриланса на FL.RU на PYTHON

In [1]:
# Подключаем библиотеку для поиска элементов кода на сайте !!!!!!!!!!!!!
# Библиотека PYTHON для извлечения данных из файлов HTML и XML
from bs4 import BeautifulSoup

# Подключаем библиотеку для парсинга сайтов !!!!!!!!!!!!!
import requests

In [1]:
# Подключаем библиотеку для поиска элементов кода на сайте !!!!!!!!!!!!!
# Библиотека PYTHON для извлечения данных из файлов HTML и XML
from bs4 import BeautifulSoup

# Подключаем библиотеки для парсинга сайтов !!!!!!!!!!!!!
import requests
import lxml
import os
import json
import csv

# Работа с файлами XLSX
import openpyxl
import xlsxwriter

# Использование объекта datetime :
from datetime import datetime
from time import sleep
import time
import random

In [2]:
# Переменная со ссылкой на сайт
url = 'https://vc.ru/popular'

In [ ]:
# Переменная со ссылкой на сайт
url = 'https://vc.ru/new'

# ФУНКЦИЯ № 1. Определение переменных

In [75]:
def variable():
# 0. Создаём словарь заголовков
    headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
        }
    
# 1. Список адресов основного хоста
    hosts = ['https://www.fl.ru']
    
# 2. Список адресов страниц для парсинга
    urls = ['https://www.fl.ru/projects/category/programmirovanie/parsing-dannyih/']
# url_2 = 'https://www.fl.ru/projects/category/programmirovanie/parsing-dannyih/page-2/'

# 3. Список имен сайтов - список словарей
    names = [{"name": u.split('//')[1].split('.')[1]} for u in urls]
    
# 4. Словарь путей для папок с данными
    type_files = ['csv', 'html', 'jpg', 'json', 'txt', 'xlsx']
    path_files = ['../data/processed/csv/', '../data/interim/html/', '../data/processed/img/',
                 '../data/processed/json/', '../data/processed/txt/', '../data/processed/xlsx/'
                ]
    extension_files = ['.csv', '.html', '.jpg', '.json', '.txt', '.xlsx']
    
    paths = [{'type_file': type_files[i], 'path_file': path_files[i], 'extension_file': extension_files[i]} for i in range(len(type_files))]
    
# 5. Словарь атрибутов Супа - список списков
    divs = ['div']
    classes = ['b-page__lenta']
    pattern = [[divs[i], classes[i]] for i in range(len(divs))]
    
    
    return headers, hosts, urls, names, paths, pattern

In [7]:
print(variable()[3][0]['name'])
# print(variable()[3][0])

vc


# ФУНКЦИЯ № 2. Сохранение HTML-кода для тестирования

In [7]:
# import os
# Отправляем запрос на страницу, передаём в качестве GET-параметра URL-адрес
# Сохраним нашу страницу в файл И ЗАКОММЕНТИРУЕМ ЭТУ КОМАНДУ за ненадобностью

def write_html(headers, url, path):
    # path = '../data/interim/html/'
# Если папки для сохранения результатов не существует, то она будет создана
    if not os.path.isdir(path[1]['path_file']):
        print(f'Fun.2.0. Folder "{path[1]['path_file']}" does NOT exist - it will be created !!!')
        os.mkdir(path[1]['path_file'])
    else:
        print(f'Fun.2.0. Folder "{path[1]['path_file']}" FOUND!!! It`s OK')
    
    print(f'Fun.2.0. START of function "WRITE_HTML()"\n')
    
    count = 1
# Проходим циклом по списку со ссылками на страницы сайтов
    for u in url:
        r = requests.get(url=u, headers=headers)
        src = r.text

        file_start = 'freelance_'
        name = u.split('//')[1].split('.')[1]
        file_name = file_start + name
        file_path = os.path.join(path[1]['path_file'], file_name) + path[1]['extension_file']
        print(f'{count}. Function "WRITE_HTML({name})" EXECUTING ...')

# Проверка существования старых файлов с таким же названием (если они существуют - обновляем)
        if not os.path.exists(file_path):
            print(f"\tThere is NO file \"{file_name}\" in the folder \"{path[1]['path_file']}\" - It will be CREATED.")
        else:
            print(f"\tIn the folder \"{path[1]['path_file']}\" the file \"{file_name}\" EXISTS.")
            print(f'\tThe file "{file_name}" will be DELETED and REWRITTEN.')
            os.remove(file_path)
            
# Сохраняем код по указанному пути
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(src)
        print(f'\tHTML-code \"{file_name}\" is SAVED in folder \"{path[1]['path_file']}\"')
        print(f'Function "WRITE_HTML({name.upper()})" is EXECUTED\n')
        count += 1

    print(f'Fun.2.0. Function "WRITE_HTML()" COMPLETED successfully!')

# Безусловный запуск функции
if __name__ == '__main__':
    write_html(variable()[0], variable()[2], variable()[4])

Fun.2.0. Folder "../data/interim/html/" FOUND!!! It`s OK
Fun.2.0. START of function "WRITE_HTML()"

1. Function "WRITE_HTML(fl)" EXECUTING ...
	There is NO file "freelance_fl" in the folder "../data/interim/html/" - It will be CREATED.
	HTML-code "freelance_fl" is SAVED in folder "../data/interim/html/"
Function "WRITE_HTML(FL)" is EXECUTED

Fun.2.0. Function "WRITE_HTML()" COMPLETED successfully!


# ФУНКЦИЯ № 3.0. Чтение HTML-кода, сохранение его в список

In [8]:
# READ_HTML:
def read_html(name, path):
    print(f'Fun.3.0. START of function "READ_HTML()"')
    count = 1
    
    # print(name)
    file_start = 'freelance_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
    print(f'{count}. Function "READ_HTML({name.upper()})" is EXECUTING...')
    
# Проверка существования файлов с HTML-кодом. Существующие коды собираем в список.
    if os.path.exists(file_path):
        print(f"\tIn the folder \"{path['path_file']}\" the file \"{file_name}\" EXISTS.")
        
        with open(file_path, encoding='utf-8') as file:
            src = file.read()
            
        print(f'\tHTML-code "{file_name}" is OPENED from {file_path}')
    else:
        print(f"\tThere is NO file \"{file_name}\" in the \"{path['path_file']}\" folder!")
        print('\tIt is necessary to START function "WRITE_HTML()" !!!\n')
        # break
        
    count += 1
    print(f'Fun.3.0. "READ_HTML({name.upper()})" COMPLETED SUCCESSFULLY!\n')

    return src

# Безусловный запуск функции
if __name__ == '__main__':
    read_html(variable()[3][0]['name'], variable()[4][1])

Fun.3.0. START of function "READ_HTML()"
1. Function "READ_HTML(FL)" is EXECUTING...
	In the folder "../data/interim/html/" the file "freelance_fl" EXISTS.
	HTML-code "freelance_fl" is OPENED from ../data/interim/html/freelance_fl.html
Fun.3.0. "READ_HTML(FL)" COMPLETED SUCCESSFULLY!



# ФУНКЦИЯ № 6.0. Проверка наличия старых CSV файлов в папке, создание заголовков

In [99]:
# ЕСЛИ ФАЙЛ УЖЕ ИМЕЕТСЯ, ТО НИЧЕГО ДЕЛАТЬ НЕ НУЖНО - PREP_CSV(path, name)
# Для подготовки CSV файла в YIELD функции - def csv_preparation(path, name)
def prep_csv(name, path):
    
    print(f'\tFun.6.0. START "PREP_CSV({name.upper()})"')
    
    file_start = 'freelance_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
        
# Проверка существования старых CSV файлов с таким же названием (если они существуют - НИЧЕГО С НИМИ НЕ ДЕЛАЕМ).
    if os.path.exists(file_path):
        print(f"\t\tThe file \"{file_name}\" already EXISTS in folder \"{path['type_file'].upper()}\".")
    else:
        print(f"\t\tThere is NO file \"{file_name}\" in the folder \"{path['path_file'].upper()}\" - It will be CREATED.\n")
                
        # Собираем заголовки для CSV-файла
        with open(file_path, 'w', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(
                (
                    'Number',
                    'Author',
                    'Date',
                    'Topic',
                    'Shorts',
                    'Link',
                    'DateTime'
                )
            )
    
    print(f'\tFun.6.0. END "PREP_CSV({name.upper()})"')

In [100]:
prep_csv('razrabotat', variable()[4][0])

	Fun.6.0. START "PREP_CSV(RAZRABOTAT)"
		There is NO file "freelance_razrabotat" in the folder "../DATA/PROCESSED/CSV/" - It will be CREATED.

	Fun.6.0. END "PREP_CSV(RAZRABOTAT)"


# ФУНКЦИЯ № 7.0. Создание CSV-файла

In [87]:
# Для заполнения CSV файла содержимым построчно - def save_csv(name, count, title, link, date_time, path):
def save_csv(name, count, title, link, date_time, path):
    
    file_start = 'freelance_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
    
# Заполняем CSV файл построчно (внутри цикла); "А" - APPEND - добавление записей 
    with open(file_path, 'a', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(
            (
                count,
                title,
                link,
                date_time
            )
        )

# ФУНКЦИЯ № 9. Проверка наличия папки для JSON-файлов, старых JSON-файлов и создание нового JSON-файла

In [97]:
def save_json(name, path, freelance_list):

# Если папки для сохранения результатов не существует, то она будет создана
    if not os.path.isdir(path['path_file']):
        print(f"\nFun.9.0. Folder \"{path['type_file'].upper()}\" does NOT exist - it will be CREATED !!!")
        os.mkdir(path['path_file'])
    else:
        print(f"\nFun.9.0 Folder \"{path['type_file'].upper()}\" FOUND!!! It`s OK")


    file_start = 'freelance_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
    print(f'Fun.9.0. START function SAVE_JSON("{name.upper()}")')
    
# Проверка существования старых JSON файлов с таким же названием (если они существуют - обновляем).
    if not os.path.exists(file_path):
        print(f"\tThere is NO file \"{file_name}\" in the folder \"{path['path_file'].upper()}\" - It will be CREATED.")
    else:
        print(f"\tIn the folder \"{path['type_file'].upper()}\" the file \"{file_name}\" EXISTS.")
        print(f'\tThe file "{file_name}" will be DELETED and REWRITTEN.')
        os.remove(file_path) # DON'T FORGET TO OPEN THE LINE !!!!!!!!!!!!!!!!!!!!!!!!!!!!

    
# Создаём третий файл в формате JSON, в который будем записывать данные в виде словаря
# который весьма полезен и удобен для чтения (в дополнение к формату CSV)
    with open(file_path, 'a', encoding='utf-8') as file:
        json.dump(freelance_list, file, indent=4, ensure_ascii=False)

    print(f"Fun.9.0. FINISH function SAVE_JSON(\"{name.upper()}\")")

In [3]:
# Создаём запрос
# response = requests.get(url)

In [55]:
# Объект SOUP
# soup = BeautifulSoup(response.text, 'html')
soup = BeautifulSoup(read_html(variable()[3][0]['name'], variable()[4][1]), 'lxml')

# Вылезает куча кода в виде текста
# print(soup)

Fun.3.0. START of function "READ_HTML()"
1. Function "READ_HTML(FL)" is EXECUTING...
	In the folder "../data/interim/html/" the file "freelance_fl" EXISTS.
	HTML-code "freelance_fl" is OPENED from ../data/interim/html/freelance_fl.html
Fun.3.0. "READ_HTML(FL)" COMPLETED SUCCESSFULLY!



In [13]:
# Определяем DIV-ы с нужной инфой
# articles = soup.find('div', class_ = 'content content--short')
# div = soup.find('div', class_ = 'pt-24 pb-32 card-lenta-new b-post  b-post_padbot_15 b-post_bordbot_eee b-post_relative')
div = soup.find('div', class_ = 'card-lenta-new')
print(div)

<div class="pt-24 pb-32 card-lenta-new b-post b-post_padbot_15 b-post_bordbot_eee b-post_relative" data-id="qa-lenta-1" id="project-item5400634" qa-project-name="project-item5400634">
<div class="card-lenta-new__item-left"></div>
<div class="d-flex align-items-center gap-8">
</div>
<div class="b-post__grid">
<h2 class="text-h5 b-post__title b-post__grid_title p-0">
<a class="text-dark text-decoration-none link-hover-danger cursor-pointer" href="/projects/5400634/razrabotat-parser-chatov-dlya-telegrama-cherez-klienstkoe-api-telegrama.html" id="prj_name_5400634" name="prj5400634" onclick="localStorage.setItem('disposableProjectId', 5400634)">разработать парсер чатов для телеграма через клиенсткое апи-телеграма</a>
</h2>
<script type="text/javascript">document.write('<div class="d-flex align-items-center b-post__price p-0 ml-lg-16 b-post__grid_price b-post__price_fontsize_15 b-post__price_bold"> <span class="text-4 text-dark text-decoration-none"> 15&nbsp;000&nbsp;<span class="fl-rub"><sp

In [59]:
# Чтобы определить степени вложенности DIV-ов применяем трюк PRETTIFY:
print(div.prettify())

<div class="pt-24 pb-32 card-lenta-new b-post b-post_padbot_15 b-post_bordbot_eee b-post_relative" data-id="qa-lenta-1" id="project-item5400634" qa-project-name="project-item5400634">
 <div class="card-lenta-new__item-left">
 </div>
 <div class="d-flex align-items-center gap-8">
 </div>
 <div class="b-post__grid">
  <h2 class="text-h5 b-post__title b-post__grid_title p-0">
   <a class="text-dark text-decoration-none link-hover-danger cursor-pointer" href="/projects/5400634/razrabotat-parser-chatov-dlya-telegrama-cherez-klienstkoe-api-telegrama.html" id="prj_name_5400634" name="prj5400634" onclick="localStorage.setItem('disposableProjectId', 5400634)">
    разработать парсер чатов для телеграма через клиенсткое апи-телеграма
   </a>
  </h2>
  <script type="text/javascript">
   document.write('<div class="d-flex align-items-center b-post__price p-0 ml-lg-16 b-post__grid_price b-post__price_fontsize_15 b-post__price_bold"> <span class="text-4 text-dark text-decoration-none"> 15&nbsp;000&n

In [171]:
# Определяем DIV-ы с нужной инфой - ВАРИАНТ С ПАТТЕРНОМ
PATTERN = variable()[5][0]
div = soup.find(PATTERN[0], class_= PATTERN[1])
# print(div)
print(div.prettify())

<div class="b-page__lenta">
 <div id="projects-list">
  <script type="text/javascript">
   var openedProjects = new Array();
    var isPrjCssOpened =true;

    /* отправка yandex metrika */
    function yaMetrikaSend(action) {
        if (typeof sendWindowYmEventsLegacy !== 'undefined') {
            sendWindowYmEventsLegacy('auto',action);
        }
    }
  </script>
  <div class="pt-24 pb-32 card-lenta-new b-post b-post_padbot_15 b-post_bordbot_eee b-post_relative" data-id="qa-lenta-1" id="project-item5401005" qa-project-name="project-item5401005">
   <div class="card-lenta-new__item-left">
   </div>
   <div class="d-flex align-items-center gap-8">
   </div>
   <div class="b-post__grid">
    <h2 class="text-h5 b-post__title b-post__grid_title p-0">
     <a class="text-dark text-decoration-none link-hover-danger cursor-pointer" href="/projects/5401005/baza-kompaniy.html" id="prj_name_5401005" name="prj5401005" onclick="localStorage.setItem('disposableProjectId', 5401005)">
      База 

In [191]:

div_1 = div.find('div', 'b-post__grid')
print(div_1.prettify())

<div class="b-post__grid">
 <h2 class="text-h5 b-post__title b-post__grid_title p-0">
  <a class="text-dark text-decoration-none link-hover-danger cursor-pointer" href="/projects/5401005/baza-kompaniy.html" id="prj_name_5401005" name="prj5401005" onclick="localStorage.setItem('disposableProjectId', 5401005)">
   База компаний
  </a>
 </h2>
 <script type="text/javascript">
  document.write('<div class="d-flex align-items-center b-post__price p-0 ml-lg-16 b-post__grid_price b-post__price_fontsize_13"> <span class="text-4 text-dark text-decoration-none"> По договоренности </span> </div>');
 </script>
 <script type="text/javascript">
  document.write('<proposal-hidden-modal id="proposal-hidden-modal"></proposal-hidden-modal> <div class="b-post__body b-post__grid_descript b-post__body_overflow_hidden b-layuot_width_full"> <div class="b-post__txt text-5 "> Нужно сформировать актуальную базу компаний(производители) из Европейкой части РФ, которые занимаются двумя видами товаров, ТЕКСТИЛЬ и Пр

In [232]:
import json
# Код для Price
# div_2 = json.loads(div_1.find('script', type='application/ld+json')) #.find('span', class_='text-4')
div_2 = div_1.find('script') #.find('span', class_='text-4')
# script = soup.find('script', attrs={'id': 'videoObject'})
# data = json.loads(str(div_2.contents[0])) #
# price = data['contentUrl']
# print(data)
print(div_2)

# data = json.loads(soup.find('script', type='application/ld+json').text)
# print(data['video']['transcript'])
# data = json.loads(str(script.contents[0]))


# Код для Shorts
# div_2 = div_1.find('script') #.find_next_sibling()

<script type="text/javascript">document.write('<div class="d-flex align-items-center b-post__price p-0 ml-lg-16 b-post__grid_price b-post__price_fontsize_13"> <span class="text-4 text-dark text-decoration-none"> По договоренности </span> </div>');</script>


In [78]:
# author__name
# author = div.find('div', class_='b-post__foot').find('span') #.text.strip()
# price = div.find('div', class_='d-flex align-items-center b-post__price p-0 ml-lg-16 b-post__grid_price b-post__price_fontsize_15 b-post__price_bold')
price = div.find('div', class_='b-post__price')
print(price)
# print(author.prettify())

None


In [77]:
# Достаём текст заголовка из этого DIV-а
# title = div.text
title = div.find('h2', class_='b-post__title').text.strip()
print(title)

разработать парсер чатов для телеграма через клиенсткое апи-телеграма


In [79]:
# Краткое содержание
shorts = div.find('div', class_='b-post__body b-post__grid_descript b-post__body_overflow_hidden b-layuot_width_full') #.text.strip()
print(shorts)
# block-text

None


In [80]:
# Ссылку ищем как элемент словаря:
host = variable()[1][0]
link = host + div.find('h2', class_='b-post__title').find('a')['href']
# link = articles.a['href']

print(link)
# print(title, link)

https://www.fl.ru/projects/5400634/razrabotat-parser-chatov-dlya-telegrama-cherez-klienstkoe-api-telegrama.html


In [19]:
# Время публикации:
time_pub = div.find('a', class_ = 'tm-article-datetime-published')
print(time_pub)

<a class="tm-article-datetime-published tm-article-datetime-published_link" href="/ru/companies/bothub/news/884024/"><!--[--><time datetime="2025-02-19T15:39:28.000Z" title="2025-02-19, 18:39">только что</time><!--]--></a>


In [20]:
# Время публикации:
# time_pub = div.find('a', class_ = 'noshrink').['time datetime']
# time_pub = div.find('div', class_ = 'content-header-details__inner').find('time')
time_dt = div.find('a', class_ = 'tm-article-datetime-published').find('time')['datetime']
time_t = div.find('a', class_ = 'tm-article-datetime-published').find('time')['title']
print(time_dt, '-', time_t)
# print(title, link)

2025-02-19T15:39:28.000Z - 2025-02-19, 18:39


## ЭТО СКАЧАННЫЙ КОД !!!

In [83]:
# Достаём все заголовки статей со ссылками (используем цикл):
# Объект SOUP
soup = BeautifulSoup(read_html(variable()[3][0]['name'], variable()[4][1]), 'lxml')
HOST = variable()[1][0]
PATTERN = variable()[5][0]

article = soup.find(PATTERN[0], class_= PATTERN[1])
divs = article.find_all('div', class_ = 'b-post')
count = 1
for art in divs:
    title = art.find('h2', class_='b-post__title').text.strip() if art.find('h2', class_='b-post__title').text.strip() else 'Absent'
    link = HOST + art.find('h2', class_='b-post__title').find('a')['href']
    date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    print(f'{count}. {title}\n{link}\n{date_time}\n')
    count +=1

Fun.3.0. START of function "READ_HTML()"
1. Function "READ_HTML(FL)" is EXECUTING...
	In the folder "../data/interim/html/" the file "freelance_fl" EXISTS.
	HTML-code "freelance_fl" is OPENED from ../data/interim/html/freelance_fl.html
Fun.3.0. "READ_HTML(FL)" COMPLETED SUCCESSFULLY!

1. разработать парсер чатов для телеграма через клиенсткое апи-телеграма
https://www.fl.ru/projects/5400634/razrabotat-parser-chatov-dlya-telegrama-cherez-klienstkoe-api-telegrama.html
2025-02-20 21:48:43

2. Парсер фото со цсетей
https://www.fl.ru/projects/5400624/parser-foto-so-tssetey.html
2025-02-20 21:48:43

3. Парсинг картинок ИИ
https://www.fl.ru/projects/5400592/parsing-kartinok-ii.html
2025-02-20 21:48:43

4. Парсер, нужно сделать генератор цифровых отпечатков  TLS,SSL
https://www.fl.ru/projects/5400581/parser-nujno-sdelat-generator-tsifrovyih-otpechatkov---tlsssl.html
2025-02-20 21:48:43

5. Парсинг данных с сайта EMobiletracker (PHP)
https://www.fl.ru/projects/5400118/parsing-dannyih-s-sayta-e

### Проникаем по ссылке вглубь сайта для получения полной информации

In [98]:
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
        }
u = 'https://www.fl.ru/projects/5400634/razrabotat-parser-chatov-dlya-telegrama-cherez-klienstkoe-api-telegrama.html'
path = variable()[4]

r = requests.get(url=u, headers=headers)
src = r.text
file_start = 'freelance_'
name = 'razrabotat'
file_name = file_start + name
file_path = os.path.join(path[1]['path_file'], file_name) + path[1]['extension_file']

# Сохраняем код по указанному пути
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(src)

In [ ]:
# url = 'https://www.fl.ru/projects/5400634/razrabotat-parser-chatov-dlya-telegrama-cherez-klienstkoe-api-telegrama.html'

#### Получение информации со страницы

In [162]:
# Дальше ОПЯТЬ работаем с сохранённым локально HTML-файлом
with open("../data/interim/html/freelance_razrabotat.html", encoding="utf-8") as file:
    src = file.read()

soup_0 = BeautifulSoup(src, "lxml")

freelance_data = soup_0.find(class_="b-layout")

In [170]:
# Пройдём в цикле по строкам таблицы
u = 'https://www.fl.ru/projects/5400634/razrabotat-parser-chatov-dlya-telegrama-cherez-klienstkoe-api-telegrama.html'
freelance_info = []
count = 1
price = soup_0.find('h2', class_='b-post__title').find_next_sibling().find_child() #.text.strip()
date = freelance_data.find('div', class_='b-layout__txt b-layout__txt_padbot_30 mt-32').find('div', class_='text-5').text.strip().split(' ')[0]
title = soup_0.find('h1', class_='text-1').text.strip()
shorts = freelance_data.find('div', class_='text-5 b-layout__txt_padbot_20').text.strip()
# shorts = freelance_data.find('div', class_='b-layout_overflow') #.text.strip()
link = u
date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

freelance_info.append(
    {
        'Number': count,
        'Price': price,
        'Date': date,
        'Topic': title,
        'Shorts': shorts,
        'Link': link,
        'DateTime': date_time
    }
)
print(f'{count}. {date} - {price}\n{title}\n{shorts}\n{link}\n{date_time}\n')

AttributeError: 'NoneType' object has no attribute 'find_next_sibling'

In [145]:
print(freelance_info)

[{'Number': 1, 'Price': None, 'Date': '20.02.2025', 'Topic': 'разработать парсер чатов для телеграма через клиенсткое апи-телеграма', 'Shorts': 'задача: мне нужно парсить по 10к-30к тг-чатов/день у меня есть покупные тг-акки  нужно разработать решение, которое будет:\xa0 1) принимать через единый эндпоинт разные запросы 2) выполнять запрос через любой подключенные аккаунт на котором не потрачен лимит (отдавать на запрос json полученные от тг без изменений) 3) хранить инфу сколько запросов выполнил каждый аккаунт, чтобы менеджерить внутренние лимиты телеграма и не обращаться больше чем можно 4) нужно чтобы новые аккаунты можно было очень легко добавлять в проект', 'Link': 'https://www.fl.ru/projects/5400634/razrabotat-parser-chatov-dlya-telegrama-cherez-klienstkoe-api-telegrama.html', 'DateTime': '2025-02-21 16:06:23'}]


## ЭТО ИНТЕРНЕТ - ONLINE !!!

In [85]:
# url_2 = 'https://www.fl.ru/projects/category/programmirovanie/parsing-dannyih/page-2/'
URL = variable()[2][0]
page=3

url_temp = URL + f'page-{page}/'
print(url_temp)

https://www.fl.ru/projects/category/programmirovanie/parsing-dannyih/page-3/


In [168]:
# Рабочий вариант без скроллинга

HEADERS = variable()[0]
HOST = variable()[1][0]
URL = variable()[2][0]
PATTERN = variable()[5][0]

page = 1
count = 1

r = requests.get(url=URL, headers=HEADERS)
src = r.text
soup = BeautifulSoup(src, 'lxml')
article = soup.find(PATTERN[0], class_= PATTERN[1])
divs = article.find_all('div', class_ = 'b-post')

print(f'PAGE={page}')
for art in divs:
    title = art.find('h2', class_='b-post__title').text.strip() if art.find('h2', class_='b-post__title').text.strip() else 'Absent'
    link = HOST + art.find('h2', class_='b-post__title').find('a')['href']
    price = art.find('h2', class_='b-post__title').find_next_sibling().find_child() #.text.strip()
    date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    sleep(random.randrange(1, 3)) # Для симуляции работы реального пользователя, а не робота
    print(f'{count}. {title}\n{link}\n{date_time}\n{price}\n')
    count +=1

PAGE=1
1. База компаний
https://www.fl.ru/projects/5401005/baza-kompaniy.html
2025-02-21 18:11:41
None

2. Парсер фото со цсетей
https://www.fl.ru/projects/5400624/parser-foto-so-tssetey.html
2025-02-21 18:11:43
None

3. Парсер, нужно сделать генератор цифровых отпечатков  TLS,SSL
https://www.fl.ru/projects/5400581/parser-nujno-sdelat-generator-tsifrovyih-otpechatkov---tlsssl.html
2025-02-21 18:11:44
None

4. Спарсить сайт
https://www.fl.ru/projects/5400787/sparsit-sayt.html
2025-02-21 18:11:45
None

5. Парсинг текстового инфо с изображений
https://www.fl.ru/projects/5400592/parsing-tekstovogo-info-s-izobrajeniy.html
2025-02-21 18:11:47
None

6. Парсинг данных с сайта EMobiletracker (PHP)
https://www.fl.ru/projects/5400118/parsing-dannyih-s-sayta-emobiletracker-php.html
2025-02-21 18:11:48
None

7. Разработка парсера ВК для фильтрации целевой аудитории через рекламный кабинет ВКонтакте
https://www.fl.ru/projects/5400503/razrabotka-parsera-vk-dlya-filtratsii-tselevoy-auditorii-cherez-r

In [159]:
# Интернет - Первая страница - С проникновением вглубь кода

HEADERS = variable()[0]
HOST = variable()[1][0]
URL = variable()[2][0]
PATTERN = variable()[5][0]

freelance_info = []

page = 1
count = 1

r = requests.get(url=URL, headers=HEADERS)
src = r.text
soup = BeautifulSoup(src, 'lxml')
article = soup.find(PATTERN[0], class_= PATTERN[1])
divs = article.find_all('div', class_ = 'b-post')

print(f'PAGE={page}')
for art in divs:
    title = art.find('h2', class_='b-post__title').text.strip() if art.find('h2', class_='b-post__title').text.strip() else 'Absent'
    # price = art.find('div', class_='d-flex align-items-center b-post__price p-0 ml-lg-16 b-post__grid_price b-post__price_fontsize_15 b-post__price_bold').find('span').text.strip() if art.find('div', class_='text-4 text-dark text-decoration-none').find('span').text.strip() else 'Absent'
    link = HOST + art.find('h2', class_='b-post__title').find('a')['href']
    date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    sleep(random.randrange(1, 3)) # Для симуляции работы реального пользователя, а не робота

# Заходим на каждую страничку
    r_temp = requests.get(url=link, headers=HEADERS)
    src_temp = r_temp.text
    soup_temp = BeautifulSoup(src_temp, 'lxml')
    
    freelance_data = soup_temp.find(class_="b-layout")
    # author = freelance_data.find('span', class_='text-4').text.strip()
    # price = freelance_data.find('div', class_='d-flex align-items-center b-post__price p-0 ml-lg-16 b-post__grid_price b-post__price_fontsize_15 b-post__price_bold').find('span').text.strip()
    date = freelance_data.find('div', class_='b-layout__txt b-layout__txt_padbot_30 mt-32').find('div', class_='text-5').text.strip().split(' ')[0]
    shorts = freelance_data.find('div', class_='text-5 b-layout__txt_padbot_20').text.strip() if freelance_data.find('div', class_='text-5 b-layout__txt_padbot_20').text.strip() else 'Absent'

    freelance_info.append(
        {
        'Number': count,
        # 'Price': price,
        'Date': date,
        'Topic': title,
        'Shorts': shorts,
        'Link': link,
        'DateTime': date_time
        }
    )
    print(f'{count}. {date} - \n{title}\n{shorts}\n{link}\n{date_time}\n')
    count +=1

PAGE=1
1. 21.02.2025 - 
Парсер, нужно сделать генератор цифровых отпечатков  TLS,SSL
Парсер, нужно сделать генератор цифровых отпечатков  TLS,SSL Прошу сразу писать - 1. хорошо ли знакомы на практике с TLS,SSL 2. как с вами связаться
https://www.fl.ru/projects/5400581/parser-nujno-sdelat-generator-tsifrovyih-otpechatkov---tlsssl.html
2025-02-21 16:22:25

2. 21.02.2025 - 
Спарсить сайт
Требуется выгрузить с данного сайта krep-komp.ru/ весь каталог полностью.  С каждой карточки выгрузить следующее:  - Наименование - Категории и подкатегории - Артикул - Стоимость - Варианты упаковки - Оба описания (то, которое идёт в виде типа таблицы, и то, которое просто в виде текста)  На скриншоте я выделил всё, что требуется выгрузить по каждому из товаров.
https://www.fl.ru/projects/5400787/sparsit-sayt.html
2025-02-21 16:22:28

3. 20.02.2025 - 
Парсер фото со цсетей
Нужно написать парсер, который будет выкачивать фотографии пользователей из социальных сетей. Подробности в файле.  Указывайте сразу

AttributeError: 'NoneType' object has no attribute 'text'

In [160]:
print(freelance_info)

[{'Number': 1, 'Date': '21.02.2025', 'Topic': 'Парсер, нужно сделать генератор цифровых отпечатков \x96 TLS,SSL', 'Shorts': 'Парсер, нужно сделать генератор цифровых отпечатков \x96 TLS,SSL Прошу сразу писать - 1. хорошо ли знакомы на практике с TLS,SSL 2. как с вами связаться', 'Link': 'https://www.fl.ru/projects/5400581/parser-nujno-sdelat-generator-tsifrovyih-otpechatkov---tlsssl.html', 'DateTime': '2025-02-21 16:22:25'}, {'Number': 2, 'Date': '21.02.2025', 'Topic': 'Спарсить сайт', 'Shorts': 'Требуется выгрузить с данного сайта krep-komp.ru/ весь каталог полностью.  С каждой карточки выгрузить следующее:  - Наименование - Категории и подкатегории - Артикул - Стоимость - Варианты упаковки - Оба описания (то, которое идёт в виде типа таблицы, и то, которое просто в виде текста)  На скриншоте я выделил всё, что требуется выгрузить по каждому из товаров.', 'Link': 'https://www.fl.ru/projects/5400787/sparsit-sayt.html', 'DateTime': '2025-02-21 16:22:28'}, {'Number': 3, 'Date': '20.02.20

### ПАГИНАЦИЯ для 5 страниц 

In [96]:
# ПАГИНАЦИЯ для 5 страниц
name = 'fl'
HEADERS = variable()[0]
HOST = variable()[1][0]
URL = variable()[2][0]
path_csv = variable()[4][0]
PATTERN = variable()[5][0]

page = 3
max_page = 5
count_link = 1
articles_list = []

while page <= max_page:
    print(f'PAGE={page}')
    url_temp = URL if (page==1) else URL + f'page-{page}/'
    # url_temp = URL + f'page-{page}/'
    r = requests.get(url=url_temp, headers=HEADERS)
    if not r:
        print('Страницы кончились !')
        break
    src = r.text
    soup = BeautifulSoup(src, 'lxml')
    article = soup.find(PATTERN[0], class_= PATTERN[1])
    divs = article.find_all('div', class_ = 'b-post')
    time.sleep(2)

    for art in divs:
        title = art.find('h2', class_='b-post__title').text.strip() if art.find('h2', class_='b-post__title').text.strip() else 'Absent'
        link = HOST + art.find('h2', class_='b-post__title').find('a')['href']
        date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        sleep(random.randrange(1, 3)) # Для симуляции работы реального пользователя, а не робота
        print(f'{count_link}. {title}\n{link}\n{date_time}\n')
        
        save_csv(name, count_link, title, link, date_time, path_csv)
        count_link +=1
    page += 1

PAGE=3
1. Написание парсера проверки регистрации Selenium или аналоги
https://www.fl.ru/projects/5388926/napisanie-parsera-proverki-registratsii-selenium-ili-analogi.html
2025-02-20 22:37:15

2. парсинг страницы LinkedIn
https://www.fl.ru/projects/5388498/parsing-stranitsyi-linkedin.html
2025-02-20 22:37:17

3. Парсинг реестра
https://www.fl.ru/projects/5387533/parsing-reestra.html
2025-02-20 22:37:18

4. Спарсить данные с сайта
https://www.fl.ru/projects/5387288/sparsit-dannyie-s-sayta.html
2025-02-20 22:37:20

5. Web-скраппер для Google Map Reviews
https://www.fl.ru/projects/5387171/web-skrapper-dlya-google-map-reviews.html
2025-02-20 22:37:22

6. Бот-парсинг в телеграмм
https://www.fl.ru/projects/5123568/bot-parsing-v-telegramm.html
2025-02-20 22:37:23

7. Написать парсер для криптовалютных пар p2p
https://www.fl.ru/projects/5386259/napisat-parser-dlya-kriptovalyutnyih-par-p2p.html
2025-02-20 22:37:25

8. Парсинг фото с сайта
https://www.fl.ru/projects/5385342/parsing-foto-s-sayta-.

In [31]:
count = 1
page=1

while page < 3:
    print(page)
    for i in range(1,13):
        print(count, end=', ')
        count +=1
    print()
    page+=1

1
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
2
13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 


# PANDAS. Обрабатываем полученные данные

In [ ]:
news = pd.DataFrame(columns=['Number', 'Author', 'Date', 'Topic', 'Shorts', 'Link', 'DateTime'])
names = variable()[3]
path_csv = variable()[4][0]
path_xls = variable()[4][5]
start = 'news_articles_'
final = 'news_articles'
file_xls = os.path.join(path_xls['path_file'], final) + path_xls['extension_file']
writer = pd.ExcelWriter(file_xls, engine='xlsxwriter')

# FORUM

In [ ]:
# Если нужно прокрутить к целевому элементу, можно воспользоваться методом .move_to_element() из ActionChains, пример:

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

driver = webdriver.Chrome()
driver.get('http://www.w3schools.com/')
target = driver.find_element_by_link_text('BROWSE TEMPLATES')
actions = ActionChains(driver)
actions.move_to_element(target)
actions.perform()

In [ ]:
# Для прокрутки в конец (начало) страницы через .send_keys():

from selenium import webdriver
from selenium.webdriver.common.keys import Keys 

driver = webdriver.Chrome()
driver.get('http://www.w3schools.com/')
driver.find_element_by_tag_name('body').send_keys(Keys.END)

# send_keys(Keys.HOME), если в начало
# так тоже сработает:
# send_keys(Keys.DOWN) send_keys(Keys.PAGE_DOWN)
# send_keys(Keys.UP) send_keys(Keys.PAGE_UP)

In [ ]:
# Для прокрутки на заданное количество пикселей метод окна .scrollBy():

scroll_vаlue = -200
scroll_by = f'window.scrollBy(0, {scroll_vаlue});'
driver.execute_script(scroll_by)

In [ ]:
# Второй вариант цикл скроллинга

from selenium.webdriver.common.keys import Keys

while True:
    # прокручиваем страницу до конца
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    # проверяем, появились ли новые товары
    if driver.find_elements(By.CLASS_NAME, "tovar"):
        with open("C:/Users/SERGEY/PycharmProjects/ТEST/Test1", "w", encoding="utf=8") as file:
            file.write(driver.page_source)
    else:
        break

In [ ]:
# Пользователь вводит название сайта и программа должна самостоятельно проверить что это за сайт: 
# Допустим пользователь вводит: google Программа должна проверить что это за сайт: google.com или google.ru и так далее... 
# И если этот сайт существует, то открыть его

import requests

domens = ['.ru', '.com', '.рф', '.net', '.org', '.ru.net', '.pro', '.ua', ]


def check_site(site_name):
    for domen in domens:
        site = f'https://www.{site_name}{domen}'
        try:
            response = requests.get(site)
            print(f'На сайт "{site}" можно перейти')
        except:
            print(f'Сайт "{site}" не открывается')


check_site('google')

In [ ]:
# Как получить результат из тега script?
import requests
import json
from bs4 import BeautifulSoup

url = 'https://www.tiktok.com/@golden_men_6/video/6803237854542712070'

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}


r = requests.get(url, headers=headers)

soup = BeautifulSoup(r.content, 'html.parser')
script = soup.find('script', attrs={'id': 'videoObject'})

data = json.loads(script.text)
video_url = data['contentUrl']
print(video_url)

In [ ]:
# Рабочий код
import requests
import json
from bs4 import BeautifulSoup

url = 'https://www.tiktok.com/@golden_men_6/video/6803237854542712070'

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}


r = requests.get(url, headers=headers)

soup = BeautifulSoup(r.content, 'html.parser')
script = soup.find('script', attrs={'id': 'videoObject'})

# NEXT !!!!
data = json.loads(script.next)
video_url = data['contentUrl']
print(video_url)

# data = json.loads(soup.find('script', type='application/ld+json').text)
# print(data['video']['transcript']) 

In [ ]:
# Как выбрать только русские слова из списка в python
import re

r = re.compile("[а-яА-Я]+")
words = ['apple', 'cat', 'яблоко', 'кот', 'building', 'машина', 'status',]
russian = [w for w in filter(r.match, words)]

In [221]:
import re
email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
def is_valid_email(email):
    return re.match(email_pattern, email) is not None

In [223]:
is_valid_email('email')

False

In [224]:
import re
ru_pattern = re.compile("[а-яА-Я]+")
def ru_text(stroka):
    return re.match(ru_pattern, stroka) is not None

In [227]:
ru_text('апр')

True

In [228]:
import re

s = '012-3456-7890'

print(re.search(r'\d+', s))

<re.Match object; span=(0, 3), match='012'>


In [ ]:
soup = BeautifulSoup(data, "html.parser")

script = soup.find("script")
inner_text_with_string = script.string
inner_text_with_content = script.contents[0]

print('inner_text_with_string', inner_text_with_string)
print('inner_text_with_content', inner_text_with_content)

In [ ]:
soup = Soup(html)
string = soup.find("script").text
string.split(".val(\"")[-1].split("\");")[0]

In [236]:
import re

from bs4 import BeautifulSoup

data = """
<body>
    <script>jQuery(window).load(function () {
      setTimeout(function(){
        jQuery("input[name=Email]").val("name@email.com");
      }, 1000);
    });</script>
</body>
"""
soup = BeautifulSoup(data, "html.parser")

script_tag = soup.find("script")
if script_tag:
  # contains all of the script tag, e.g. "jQuery(window)..."
  script_tag_contents = script_tag.string

  # from there you can search the string using a regex, etc.
  email = re.search(r'\.+val\("(.+)"\);', script_tag_contents).group(1)
  print(email)

name@email.com


In [241]:
import re

from bs4 import BeautifulSoup

data = """
<script type="text/javascript">document.write('<div class="d-flex align-items-center b-post__price p-0 ml-lg-16 b-post__grid_price b-post__price_fontsize_13"> <span class="text-4 text-dark text-decoration-none"> По договоренности </span> </div>');</script>
"""
soup = BeautifulSoup(data, "html.parser")

script_tag = soup.find("script")

script_tag = soup.find("script")
print(script_tag)
print()
print(script_tag.string)
print()
print(script_tag.contents[0])

# if script_tag:
#   # contains all of the script tag, e.g. "jQuery(window)..."
#   script_tag_contents = script_tag.string

#   # from there you can search the string using a regex, etc.
#   email = re.search(r'[а-яА-Я]+', script_tag_contents).group(1)
#   print(email)

# r = re.compile("[а-яА-Я]+")

<script type="text/javascript">document.write('<div class="d-flex align-items-center b-post__price p-0 ml-lg-16 b-post__grid_price b-post__price_fontsize_13"> <span class="text-4 text-dark text-decoration-none"> По договоренности </span> </div>');</script>

document.write('<div class="d-flex align-items-center b-post__price p-0 ml-lg-16 b-post__grid_price b-post__price_fontsize_13"> <span class="text-4 text-dark text-decoration-none"> По договоренности </span> </div>');



IndexError: list index out of range

In [245]:
data = """
<script type="text/javascript">document.write('<div class="d-flex align-items-center b-post__price p-0 ml-lg-16 b-post__grid_price b-post__price_fontsize_13"> <span class="text-4 text-dark text-decoration-none"> По договоренности </span> </div>');</script>
"""

soup = BeautifulSoup(data, "html.parser")

pattern = re.compile("[а-яА-Я]+")
script = soup.find("script", text=pattern)

# print(pattern.search(script.text).group(1))

# from there you can search the string using a regex, etc.
script_tag_contents = script.string
email = re.search(pattern, script_tag_contents).group(1)
print(email)

C:\Users\User\AppData\Local\Temp\ipykernel_8204\54702031.py:8: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script = soup.find("script", text=pattern)


IndexError: no such group